In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
import torchvision
import torchvision.transforms as transforms
import numpy as np

trans = torchvision.transforms.Compose([
                                        torchvision.transforms.ToTensor(),
                                        torchvision.transforms.Normalize((0.5,),(0.5,))
])

train_set = torchvision.datasets.MNIST(
    root="/data",
    download = True,
    train = True,
    transform = trans
    #transform=transforms.ToTensor()
)
test_set = torchvision.datasets.MNIST(
    root="/data",
    download = True,
    train = False,
    transform=transforms.ToTensor()
)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)

print(len(train_loader))


In [ ]:
class convNet(nn.Module) :
    def __init__(self):
        super(convNet, self).__init__()

        self.conv01 = nn.Conv2d(
            in_channels=1, 
            out_channels=32, 
            kernel_size=(3,3), 
            stride=1, 
            padding=1)
        
        self.conv02 = nn.Conv2d(
            in_channels=32, 
            out_channels=64, 
            kernel_size=3,
            stride = 1,
            padding = 1)
        
        self.dropOutLayer = nn.Dropout2d(p=0.2, inplace=False)
        self.relu = nn.ReLU()
        self.maxpool2d = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc = nn.Linear(7*7*64, 10)

    def forward(self, x):
        out = self.conv01(x)
        out = self.relu(out)
        out = self.maxpool2d(out)
        out = self.conv02(out)
        out = self.relu(out)
        out = self.maxpool2d(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


model = convNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= 1e-3)


In [ ]:
for epoch in range(3) :
    avg_cost = 0

    for X, Y in train_loader : #X는 미니배치, Y는 레이블
        X = X.to(device)
        Y= Y.to(device)
        optimizer.zero_grad()
        cost = criterion(model(X), Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / len(train_loader)
    
    print(f"Epoch = {epoch}  loss = {avg_cost}")
        

In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = test_set.test_data.view(len(test_set), 1, 28, 28).float().to(device)
    Y_test = test_set.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    print(correct_prediction)
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())